In [125]:
%pip install pandas
%pip install openpyxl
%pip install yfiles_jupyter_graphs


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [126]:
import pandas as pd
import os
from yfiles_jupyter_graphs import GraphWidget


In [127]:
def prepare_raw_data(df):

  EPOCH_ROW = 0
  CYCLE_ROW = 1
  CYCLE_RULE_ROW = 2
  VISIT_LABEL_ROW = 3
  CONDITION_ROW = 4
  VISIT_WINDOW_ROW = 5
  FIRST_ACTIVITY_ROW = 7

  FIRST_VISIT_COL = 3

  tp_id = 1
  timepoints = []
  visits = []
  activities = []
  activity_bc = {}
  activity_row = []
  result = {}
  cycles = []

  activity = None
  for index, col_def in df.iterrows():
    #print("ACT1:", index, df.iloc[index, 1])
    if index >= 7:
      activity_present = not pd.isnull(df.iloc[index, 1])
      if activity_present:
        activity = df.iloc[index, 1]
        activities.append(activity)
      activity_row.append({ 'name': activity, 'present': activity_present })
      bc_present = not pd.isnull(df.iloc[index, 2])
      if bc_present:
        if not activity in activity_bc:
          activity_bc[activity] = { 'bc': [] }  
        activity_bc[activity]['bc'].append(df.iloc[index, 2])
  #print("activities:", activities)
  #print("activity_bc:", activity_bc)
  #print("activity_row", activity_row)
  result = { 
    'activities': activities,
    'activity_bc_map': activity_bc,
  #  'activity_row_map': activity_row,
  }

  in_cycle = False
  prev_cycle = None
  for index in range(df.shape[1]):
    if index >= FIRST_VISIT_COL:
      anchor = False
      
      condition_type = None
      label = df.iloc[VISIT_LABEL_ROW, index]
      if pd.isnull(df.iloc[CONDITION_ROW,index]):
        condition_info = ""
        condition_type = "unknown"
        condition = ""
      else:
        condition_info = df.iloc[CONDITION_ROW, index]
        condition_parts = condition_info.split(":")
        #print ("CPs:", condition_parts)
        if condition_parts[0].upper() == "A":
          anchor = True
          condition_type = "anchor"
        if condition_parts[0].upper() == "P":
          condition_type = "previous"
        elif condition_parts[0].upper() == "N":
          condition_type = "next"
        if len(condition_parts) == 2:
          condition = condition_parts[1].strip()
        else:
          condition = ""
      
      # Cycle Information
      tp_index = len(timepoints) - 1
      cycle = df.iloc[CYCLE_ROW, index]
      cycle_rule = df.iloc[CYCLE_RULE_ROW, index]
      cycle_is_null = pd.isnull(df.iloc[CYCLE_ROW,index])
      #print("CY:", cycle, cycle_rule, cycle_is_null)
      if cycle_is_null:
        cycle = ""
        cycle_rule = ""
        if in_cycle:
          print("CY1a:", tp_index)
          cycles.append({ 'start_index': cycle_start, 'end_index': tp_index, 'condition': prev_cycle })
          timepoints[tp_index - 1]['cycle_index'] = len(cycles) - 1
          in_cycle = False
          prev_cycle = cycle
      else:
        cycle = str(cycle)
        #print("CY2:", cycle, prev_cycle, in_cycle)
        if ".." in cycle:
          #print("CY3:")
          if not in_cycle:
            #print("CY3a:")
            in_cycle = True
            cycle_start = tp_index + 1
          elif prev_cycle == cycle:
            #print("CY3b:")
            pass
          else:
            print("CY3c:", tp_index)
            cycles.append({ 'start_index': cycle_start, 'end_index': tp_index, 'condition': prev_cycle })
            timepoints[tp_index - 1]['cycle_index'] = len(cycles) - 1
            cycle_start = tp_index + 1
          prev_cycle = cycle
        else:
          #print("CY4:")
          cycle = ""
          cycle_rule = ""
          if in_cycle:
            print("CY4a:", tp_index)
            cycles.append({ 'start_index': cycle_start, 'end_index': tp_index, 'condition': prev_cycle })
            timepoints[tp_index - 1]['cycle_index'] = len(cycles) - 1
            in_cycle = False
            prev_cycle = cycle

      #print("C:", condition_type)
      window = df.iloc[VISIT_WINDOW_ROW, index]
      window_is_null = pd.isnull(df.iloc[VISIT_WINDOW_ROW,index])
      if window_is_null:
        window = ""
      visits.append({ 'name': label, 'window': window })
      cycle_index = None
      timepoints.append({ 'name': "tp_%s" % (tp_id), 'condition_type': condition_type, 'condition': condition, 'anchor': anchor, 'cycle_index': None })
      tp_id += 1
  print("timepoints:", timepoints)
  #print("visits:", visits)
  print("cycles:", cycles)
  result['timepoints'] = timepoints
  result['visits'] = visits
  result['cycles'] = cycles
  
  tp_activity = {}
  for tp in timepoints:
    tp_name = tp['name']
    tp_activity[tp_name] = {}
    for activity in activities:
      tp_activity[tp_name][activity] = False
  for index in range(df.shape[1]):
    if index >= FIRST_VISIT_COL:
      column = df.iloc[:, index]
      row = 0
      for x in column:
        if row >= FIRST_ACTIVITY_ROW:
          if not pd.isnull(x):
            #print("X ROW:", index, row, x)
            if x.upper() == "X":
              if activity_row[row - FIRST_ACTIVITY_ROW]['present']:
                activity = activity_row[row - FIRST_ACTIVITY_ROW]['name']
                tp = timepoints[index - FIRST_VISIT_COL]['name']
                tp_activity[tp][activity] = True
        row += 1
  #print("tp_activity:", tp_activity)
  result['timepoint_activity_map'] = tp_activity
  return result

In [128]:
def prepare_nodes_and_edges(raw_data):

  ENTRY_NODE_INDEX = 0
  EXIT_NODE_INDEX = 99999

  nodes = [
    {'id': ENTRY_NODE_INDEX, 'properties': {'label': 'Entry', 'node_type': 'entry_exit'}},
    {'id': EXIT_NODE_INDEX, 'properties': {'label': 'Exit', 'node_type': 'entry_exit'}}
  ]
  edges = []
  node_index = 1
  edge_index = 0
  prev_node = None
  activity_nodes = {}
  tp_nodes = {}

  # Add cycle condition nodes
  cycle_nodes = []
  for cycle in raw_data['cycles']:
    cycle_nodes.append(node_index)
    nodes.append({ 'id': node_index, 'properties': {'label': cycle['condition'], 'node_type': 'condition' }})
    node_index += 1
    
  # Add base timeline, the timepoints
  for tp in raw_data['timepoints']:
    label = tp['name']
    node_type = "timepoint"
    if tp['anchor']:
      label = "%s : ANCHOR" % (tp['name'])
      node_type = "anchor"
    nodes.append({ 'id': node_index, 'properties': {'label': label, 'node_type': node_type }})
    tp_nodes[tp['name']] = node_index
    if prev_node == None:
      edges.append( {id: edge_index, 'start': ENTRY_NODE_INDEX, 'end': node_index, 'properties': {'label': "entry"}})
      edge_index += 1
      prev_node = node_index
    else:
      edges.append( {id: edge_index, 'start': prev_node, 'end': node_index, 'properties': {'label': "next"}})
      edge_index += 1
      prev_node = node_index
      if not tp['cycle_index'] == None:
        cycle_node = cycle_nodes[tp['cycle_index']]
        edges.append( {id: edge_index, 'start': node_index, 'end': cycle_node, 'properties': {'label': "next"}})
        edge_index += 1
        prev_node = cycle_node
    node_index += 1
  edges.append( {id: edge_index, 'start': prev_node, 'end': EXIT_NODE_INDEX, 'properties': {'label': "exit"}})
  edge_index += 1
  
  # Add cycle condition edges
  for index, cycle in enumerate(raw_data['cycles']):
    cycle_node = cycle_nodes[index]
    start_tp_index = cycle['start_index']
    end_tp_index = cycle['end_index']
    tp = raw_data['timepoints'][start_tp_index]
    tp_node = tp_nodes[tp['name']]
    print("CYCLE:", cycle_node, start_tp_index, end_tp_index, tp_node, nodes[tp_node])
    edges.append( {id: edge_index, 'start': cycle_node, 'end': tp_node, 'properties': {'label': cycle['condition']}})
    print("EDGES", edges[-1])
    print("NODES:", nodes)
    edge_index += 1

  # Add conditions between timepoint nodes
  for index, tp in enumerate(raw_data['timepoints']):
    start_name = tp['name']
    start_index = tp_nodes[start_name]
    if tp['condition_type'] == "next":
      end_name = raw_data['timepoints'][index + 1]['name']
      end_index = tp_nodes[end_name]
      edges.append( {id: edge_index, 'start': start_index, 'end': end_index, 'properties': {'label': tp['condition']}})
    elif tp['condition_type'] == "previous":
      end_name = raw_data['timepoints'][index - 1]['name']
      end_index = tp_nodes[end_name]
      edges.append( {id: edge_index, 'start': start_index, 'end': end_index, 'properties': {'label': tp['condition']}})
    edge_index += 1

  # Add links to visits
  for index, tp in enumerate(raw_data['timepoints']):
    visit_index = node_index
    visit = raw_data['visits'][index]
    nodes.append({ 'id': visit_index, 'properties': { 'label': "%s, %s" % (visit['name'], visit['window']), 'node_type': 'visit' }})
    node_index += 1
    edges.append( {id: edge_index, 'start': tp_nodes[tp['name']], 'end': visit_index, 'properties': {'label': "contains_visit"}})
    edge_index += 1

  # Add links to activities (a fresh copy for each, not shared, makes the diagram better to understand)
  for tp, value in raw_data['timepoint_activity_map'].items():
    for activity, required in value.items():
      if required:
        activity_index = node_index
        nodes.append({ 'id': activity_index, 'properties': { 'label': activity, 'node_type': 'activity' }})
        #activity_nodes[activity] = activity_index
        node_index += 1
        if activity in raw_data['activity_bc_map']:
          for bc in raw_data['activity_bc_map'][activity]['bc']:
            bc_index = node_index
            nodes.append({ 'id': bc_index, 'properties': { 'label': bc, 'node_type': 'bc' }})
            node_index += 1
            edges.append( {id: edge_index, 'start': activity_index, 'end': bc_index, 'properties': {'label': "contains_bc"}})
            edge_index += 1
        edges.append( {id: edge_index, 'start': tp_nodes[tp], 'end': activity_index, 'properties': {'label': "contains_activity"}})
        edge_index += 1
  return nodes, edges



In [129]:
def custom_node_color(index: int, node: dict):
  if 'node_type' in node['properties']:
    if node['properties']['node_type'] == 'entry_exit':
      return 'black'
    elif node['properties']['node_type'] == 'anchor':
      return '#999999'
    elif node['properties']['node_type'] == 'condition':
      return '#999999'
    elif node['properties']['node_type'] == 'timepoint':
      return '#555555'
    elif node['properties']['node_type'] == 'visit':
      return '#c1141a'
    elif node['properties']['node_type'] == 'activity':
      return '#1555bd'
    elif node['properties']['node_type'] == 'bc':
      return '#c0d6e4'
    else:
      return 'white'
  else: 
    return 'white'

def custom_node_style(index: int, node: dict):
  if 'node_type' in node['properties']:
    if node['properties']['node_type'] == 'entry_exit':
      return { 'shape': 'round-rectangle' }
    elif node['properties']['node_type'] == 'anchor':
      return { 'shape': 'hexagon2' }
    elif node['properties']['node_type'] == 'condition':
      return { 'shape': 'diamond' }
    elif node['properties']['node_type'] == 'timepoint':
      return { 'shape': 'ellipse' }
    elif node['properties']['node_type'] == 'visit':
      return { 'shape': 'circle' }
    elif node['properties']['node_type'] == 'activity':
      return { 'shape': 'circle' }
    elif node['properties']['node_type'] == 'bc':
      return { 'shape': 'circle' }
    else:
      return { 'shape': 'circle' }
  else: 
    return { 'shape': 'circle' }


In [130]:
notebook_path = os.path.abspath("notebook.ipynb")
file_path = os.path.join(os.path.dirname(notebook_path), "source_data/berber_1.xlsx")
#file_path = os.path.join(os.path.dirname(notebook_path), "source_data/Roche Phase 3 NCT04320615.xlsx")
df = pd.read_excel(file_path, header=None)
raw_data = prepare_raw_data(df)
nodes, edges = prepare_nodes_and_edges(raw_data)

CY3c: 7
CY1a: 8
timepoints: [{'name': 'tp_1', 'condition_type': 'next', 'condition': '0..30 Days', 'anchor': False, 'cycle_index': None}, {'name': 'tp_2', 'condition_type': 'anchor', 'condition': '', 'anchor': True, 'cycle_index': None}, {'name': 'tp_3', 'condition_type': 'previous', 'condition': '+14 Days', 'anchor': False, 'cycle_index': None}, {'name': 'tp_4', 'condition_type': 'previous', 'condition': '+N Days', 'anchor': False, 'cycle_index': None}, {'name': 'tp_5', 'condition_type': 'previous', 'condition': '+14 Days', 'anchor': False, 'cycle_index': None}, {'name': 'tp_6', 'condition_type': 'previous', 'condition': '+N Days', 'anchor': False, 'cycle_index': None}, {'name': 'tp_7', 'condition_type': 'previous', 'condition': '+14 Days', 'anchor': False, 'cycle_index': 0}, {'name': 'tp_8', 'condition_type': 'previous', 'condition': '+N Days', 'anchor': False, 'cycle_index': 1}, {'name': 'tp_9', 'condition_type': 'previous', 'condition': '+N Days', 'anchor': False, 'cycle_index': No

In [131]:
w = GraphWidget()
w.orthogonal_layout()
w.set_directed(True)

w.set_nodes(nodes)
w.set_edges(edges)

w.set_node_color_mapping(custom_node_color)
w.set_node_styles_mapping(custom_node_style)
w

/Users/daveih/Library/Python/3.10/lib/python/site-packages/jupyter_client/session.py:718: UserWarning: Message serialization failed with:
keys must be str, int, float, bool or None, not builtin_function_or_method
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant
  content = self.pack(content)


GraphWidget(layout=Layout(height='500px', width='100%'))